In [ ]:
/**
 https://www.bambooweekly.com/government-corruption/ 
 https://www.bambooweekly.com/government-corruption-436/ 
 https://github.com/JoergEm/Bamboo-Weekly/tree/main 
*/

In [ ]:
// Imports
#r "nuget: Pandas.NET"
#r "nuget: ExcelDataReader"
#r "nuget: ExcelDataReader.DataSet"
using System;
using System.IO;
using System.Net.Http;
using System.Threading.Tasks;
using ExcelDataReader;
using PandasNet;

In [ ]:
// Function creating local folders
string dataFolder = "data";
string resultsFolder = "results";

if (!Directory.Exists(dataFolder))
{
    Directory.CreateDirectory(dataFolder);
    Console.WriteLine($"Created folder: {dataFolder}");
}
else
{
    Console.WriteLine($"Folder already exists: {dataFolder}");
}

if (!Directory.Exists(resultsFolder))
{
    Directory.CreateDirectory(resultsFolder);
    Console.WriteLine($"Created folder: {resultsFolder}");
}
else
{
    Console.WriteLine($"Folder already exists: {resultsFolder}");
}

In [ ]:
// Function downloading data locally
async Task DownloadExcelFile(string url, string destinationPath)
{
    using (HttpClient client = new HttpClient())
    {
        try
        {
            var response = await client.GetAsync(url);
            response.EnsureSuccessStatusCode();
            var fileBytes = await response.Content.ReadAsByteArrayAsync();
            await File.WriteAllBytesAsync(destinationPath, fileBytes);
            Console.WriteLine($"Downloaded file to: {destinationPath}");
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error downloading file: {ex.Message}");
        }
    }
}

In [ ]:
// Links and folders to recieve data and read into DataFrame
string url = "https://images.transparencycdn.org/images/CPI2022_GlobalResultsTrends.xlsx";
string filePath = Path.Combine("data", "CPI2022_GlobalResultsTrends.xlsx");
await DownloadExcelFile(url, filePath);

In [ ]:
DataFrame ReadExcelSheet(string filePath, string sheetName, int rowsToSkip)
{
    System.Text.Encoding.RegisterProvider(System.Text.CodePagesEncodingProvider.Instance);
    
    using (var stream = File.Open(filePath, FileMode.Open, FileAccess.Read))
    {
        using (var reader = ExcelReaderFactory.CreateReader(stream))
        {
            var result = reader.AsDataSet(new ExcelDataSetConfiguration()
            {
                ConfigureDataTable = (_) => new ExcelDataTableConfiguration()
                {
                    UseHeaderRow = true
                }
            });
            var dataTable = result.Tables[sheetName];
            for (int i = 0; i < rowsToSkip; i++)
            {
                dataTable.Rows.RemoveAt(0);
            }
            var dataFrame = new DataFrame();

            foreach (DataColumn column in dataTable.Columns)
            {
                var columnData = new PrimitiveDataFrameColumn<string>(column.ColumnName, dataTable.Rows.Count - rowsToSkip);
                for (int i = 0; i < dataTable.Rows.Count - rowsToSkip; i++)
                {
                    columnData[i] = dataTable.Rows[i + rowsToSkip][column].ToString();
                }
                dataFrame.Columns.Add(columnData);
            }
            return dataFrame;
        }
    }
}

var df = ReadExcelSheet(filePath, "CPI 2022 (final)", 2);

In [ ]:
// According to Transparency International, what five countries were least corrupt in 2022?
Console.WriteLine(df.Head(5));

In [ ]:
// According to the same data, what five countries were most corrupt in 2022?
Console.WriteLine(df.Tail(5));